In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Testing files/TCGAbiolinks/mrna_filtered.csv')
train.head()

,bcr_patient_barcode,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,TCGA-A8-A075-01,1.0247,1.3797,-0.4226,0.6259,0.6799,-0.2258,-0.4871,-0.7646,1.0828,...,-0.6426,-0.2856,-0.3675,-0.5557,-1.0815,-0.7442,-0.9931,-0.0280,-0.5203,-0.5849
1,TCGA-A8-A08O-01,0.3520,0.0570,0.3933,0.6852,0.5280,-0.2258,-0.4871,-0.7646,1.7885,...,-0.6426,-0.2856,-0.3675,-1.0605,-0.3078,0.2399,0.6208,0.8279,0.9387,-0.5849
2,TCGA-A8-A0A6-01,0.6152,2.0578,0.0965,-0.8106,-0.6237,-0.2258,-0.4871,-0.4277,-0.7037,...,-0.6426,-0.2856,-0.3675,0.2533,1.5514,0.9728,0.0960,-0.7322,-0.5203,0.3760
3,TCGA-AQ-A04H-01,2.4979,-1.6194,-1.2335,2.0037,-1.0397,-0.2258,-0.3148,-0.7646,-0.7037,...,-0.6426,-0.2856,1.5518,1.7588,-0.4590,0.1816,-0.0998,-0.7322,-0.5203,-0.5849
4,TCGA-BH-A0AZ-01,0.8400,1.0882,0.8629,-0.8106,1.9589,-0.2258,-0.2183,1.5879,0.9336,...,0.3537,-0.2856,-0.3675,0.7533,-1.0815,0.4864,-0.2746,-0.7322,1.2345,-0.5849


In [ ]:
target=train['bcr_patient_barcode']

In [ ]:
train.drop(['bcr_patient_barcode'], axis=1, inplace=True)
train.head()

In [ ]:
train.isnull().sum().sum()

In [ ]:
train = train.dropna(how = 'all', axis = 1)

In [ ]:
train = train.apply(lambda x : x.fillna(x.mean()), axis = 0)

In [ ]:
train.isnull().sum().sum()

0

In [ ]:
#train = train.apply(lambda x : x.fillnan(x.mean()), axis = 0)

In [ ]:
train_scaled = minmax_scale(train, axis = 0)

In [ ]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 24

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

In [ ]:
encoded_train

In [ ]:
autoencoder.summary()

In [ ]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean() > 0.95])
encoded_train.shape

In [ ]:
encoded_train

In [ ]:
#encoded_train.to_excel('/content/drive/MyDrive/Testing files/TCGAbiolinks/dae_expression.xlsx', index=False)